# Load required packages

In [ ]:
# ! pip install statsmodels # run this if you don't have statsmodels installed

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings('ignore') # this is to clear up some warning messages that might pop up

# Fancier plot tool
import seaborn as sns
sns.set_theme(context='notebook', style='whitegrid', palette='deep', font='sans-serif', font_scale=3, color_codes=True,rc={'figure.figsize':(15,10)})

In [ ]:
np.random.seed(42)

In [ ]:
_DATA_DIR = '/classes/20800_winter2024/Data'

# Linear regression (orange juice)

In [ ]:
# read data
oj = pd.read_csv('%s/oj.csv'%(_DATA_DIR))

In [ ]:
oj.head()

In [ ]:
# Generate the log price and log sales
oj['log_price'] = oj['price'].apply(lambda x: np.log(x))
oj['log_sales'] = oj['sales'].apply(lambda x: np.log(x))

In [ ]:
oj

## Visualization

In [ ]:
plt.figure(figsize = (16,8))

plt.subplot(1, 2, 1)
sns.histplot(oj, x="price");
plt.title('Histgram for price')

plt.subplot(1, 2, 2)
sns.histplot(oj, x="log_sales");
plt.xlabel('log sales')
plt.title('Histgram for loan amount')

plt.show()
#plt.savefig('histogram_oj.pdf')

In [ ]:
plt.figure(figsize = (11,8))

sns.boxplot(x="brand",y="log_price",hue="feat",data=oj,palette="Set3")
plt.ylabel('log price')
plt.title('Conditional Bar plots for log price')
plt.show()
#plt.savefig('box_oj.pdf')

In [ ]:
plt.figure(figsize = (11,8))

sns.scatterplot(x='log_price',y='log_sales',hue='brand', data = oj)
plt.xlabel('log price')
plt.ylabel('log sales')
plt.title('Scatter plot for log price vs log sales')
plt.show()
#plt.savefig('scatter_oj.pdf')

## Price Elasticity

In [ ]:
reg = smf.glm(formula='log_sales ~ log_price + brand', data=oj).fit()
print(reg.summary())

In [ ]:
# regression coefficients
reg.params

## The Design Matrix

In [ ]:
from patsy import dmatrix

x=pd.DataFrame(dmatrix(' ~ log_price + brand', data = oj)[[99, 199, 299,399,499,599]])
x.columns=['Intercept','brand[T.minute.maid]','brand[T.tropicana]','log_price']
x

## Interaction

In [ ]:
reg_interact = smf.glm(formula='log_sales ~ log_price * brand', data=oj).fit()
print(reg_interact.summary()) 

## Advertisements

We could model the additive effect on log sales volume

$$E[\log({\tt v})] = \alpha_{b} + 
{1_{[{\tt feat}]}\alpha_{\tt feat}} + \beta_b\log({\tt p}) $$

Or this and its effect on elasticity

$$E[\log(v)] = \alpha_{b} + \beta_b\log({\tt p}) + 
{1_{[{\tt feat}]}\left(\alpha_{\tt feat} 
+ \beta_{\tt feat}\log({\tt p})\right)}$$

Or its brand-specific effect on elasticity

$$E[\log(v)] = \alpha_{b} + \beta_b\log({\tt p}) + 
{1_{[{\tt feat}]}\left(\alpha_{b,\tt feat} 
+ \beta_{b,\tt feat}\log({\tt p})\right)}$$

See below for all three models

In [ ]:
# Add the intercation for 3 kinds variables
reg_full = smf.glm(formula='log_sales ~ log_price * brand * feat', data=oj).fit()
print(reg_full.summary())

## Brand-specific Elasticities

In [ ]:
# Construct the elasticities table
b = reg_full.params

data = [['Not featured & Dominicks', b["log_price"]], 
        ['Not featured & Minute Maid', b["log_price"] + b["log_price:brand[T.minute.maid]"]],
        ['Not featured & Tropicana', b["log_price"] + b["log_price:brand[T.tropicana]"]],
        ['Featured & Dominicks', b["log_price"] + b["log_price:feat"]], 
        ['Featured & Minute Maid', b["log_price"] + b["log_price:brand[T.minute.maid]"] + b["log_price:feat"] + b["log_price:brand[T.minute.maid]:feat"]],
        ['Featured & Tropicana', b["log_price"] + b["log_price"] + b["log_price:brand[T.tropicana]"] + b["log_price:feat"] + b["log_price:brand[T.tropicana]:feat"]]]

pd.DataFrame(data, columns = ['brand&feature', 'elastisity']).round(1)

## Confounding

In [ ]:
# table explaining why ads confounded our brand elasticity estimates
from statsmodels.graphics.mosaicplot import mosaic

oj_1 = pd.read_csv('%s/oj_1.csv'%(_DATA_DIR))
oj_1['move'] = oj_1.logmove.apply(lambda x: np.exp(x))
oj_1['sales_predict'] = reg_full.predict()

salestable = oj_1.pivot_table(aggfunc=np.sum, values='move', index=['feat', 'brand']).to_dict()
plt.figure(figsize = (8,8))
mosaic(salestable['move'])
plt.title('Mosaic plot of the amount of advertisement by brand')
plt.show()
#plt.savefig('mosaic_oj.pdf')

## Estimation and Goodness of Fit

In [ ]:
print("oj null deviance:",round(reg_full.null_deviance,2))
print("oj deviance:",round(reg_full.deviance,2))
print("oj R2:",round(1-reg_full.deviance/reg_full.null_deviance,2))

## Visualization of $\hat{y}$ vs $y$

In [ ]:
sns.scatterplot(x='logmove',y='sales_predict',hue='brand', data = oj_1)
logmove_linspace = np.linspace(oj_1['logmove'].min(), oj_1['logmove'].max(), 100)

plt.plot(oj_1['sales_predict'], oj_1['sales_predict'], 'black')
plt.xlabel('log sales')
plt.ylabel('predicted sales')
plt.title('fit plot for OJ linear regression')
# plt.savefig('regression_oj.pdf')

## Prediction

In [ ]:
pd.DataFrame({'predict':reg_full.predict(oj[0:10]), 'true':oj_1.loc[0:9,'logmove']}).round(2)

# Logistic Regression (default example)

In [ ]:
# read data
default = pd.read_csv('%s/default.csv'%(_DATA_DIR),index_col=0)

default.head()

In [ ]:
## fit the full model
all_columns = " + ".join(default.columns.difference(["probability"]))

my_formula = "probability~" + all_columns

proba = smf.glm(formula=my_formula , data=default, family=sm.families.Binomial()).fit()
print(proba.summary())

## Estimation and Goodness of Fit

In [ ]:
print("default null deviance:",round(proba.null_deviance,2))
print("default deviance:",round(proba.deviance,2))
print("default R2:",round(1-proba.deviance/proba.null_deviance,2))

## Visualization of $\hat{y}$ vs $y$

In [ ]:
fitted = proba.fittedvalues.values
default_proba = default['probability']

sns.boxplot(x=default_proba,y=fitted)

plt.xlabel('default category')
plt.ylabel('fitted probability of default')
plt.title('fit plot for default logistic regression')
#plt.savefig('regression_default.pdf')

## Prediction

In [ ]:
pd.DataFrame({'predict':proba.predict(default[0:9]), 'true':default.loc[0:9,'probability']}).round(2)

## Out of sample prediction

In [ ]:
import random
leaveout = random.sample(range(len(default)),1000)

# train the model WITHOUT these observations
probatrain = smf.glm(formula=my_formula, data=default.drop(leaveout), family=sm.families.Binomial()).fit()

X_test = default.loc[leaveout].drop(columns = 'probability')
# predicted probability of default on the left out data
pdefault = probatrain.predict(X_test)

In [ ]:
predict = pdefault
default_proba = default.loc[leaveout,'probability']

sns.boxplot(x=default_proba,y=predict)

plt.xlabel('default category')
plt.ylabel('fitted probability of default')
plt.title('fit plot for default logistic regression (OOS)')
#plt.savefig('oos_regression_default.pdf')

In [ ]:
def deviance(y, pred, family):
    if family == 'gaussian':
        return sum((y - pred)**2)
    if family == 'binomial':
        return -2 * sum(y * np.log(pred) + (1-y) * np.log(1-pred))
    
y_test = default.loc[leaveout,'probability']
y_train = default.drop(leaveout).probability

dev0 = deviance(y_test,np.repeat(np.mean(y_train),len(y_test)),family="binomial")
dev = deviance(y_test,np.array(pdefault),family="binomial")
R2 = 1-dev/dev0



In [ ]:
print("oos dev0:",round(dev0,4))
print("oos dev:",round(dev,4))
print("oos R2:",round(R2,4))